In [14]:
import sys


path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )

import util as du
du.add_to_path( path )

Path [/var/genie-in-the-box/src/lib] already in sys.path


In [8]:
import time

class Stopwatch:

    def __enter__( self ):

        self.start = time.time()

        return self

    def __exit__( self, *args ):

        self.end      = time.time()
        self.interval = int( ( self.end - self.start ) * 1000 )

        print( f'Elapsed time: {self.interval} milliseconds' )

with Stopwatch() as sw:

    time.sleep( 1 )

Elapsed time: 1001 milliseconds


In [22]:
import os
import glob
import json
import openai
import numpy as np

class SolutionSnapshot:

    def __init__( self, problem, solution_summary="", code="", programming_language="python", language_version="3.10", embedding=[], solution_directory="/src/conf/long-term-memory/solutions/" ):

        self.problem              = problem
        self.embedding            = embedding
        self.solution_summary     = solution_summary
        self.code                 = code
        self.programming_language = programming_language
        self.language_version     = language_version
        self.solution_directory   = solution_directory

        self._generate_embedding()
    
    def _generate_embedding( self ):

        with Stopwatch() as sw:
            response = openai.Embedding.create(
                input=self.problem,
                model="text-embedding-ada-002"
            )
        self.embedding = response[ "data" ][ 0 ][ "embedding" ]

        # arr = np.array( self.embedding )
        # print( f"Embedding shape: {arr.shape}" )
        # print( f"Embedding len: {len( self.embedding )}" )

    def compare_embedding( self, other_snapshot ):

        if not self.embedding or not other_snapshot.embedding:
            raise ValueError( "Both snapshots must have an embedding to compare." )

        return np.dot( self.embedding, other_snapshot.embedding )

    def to_json( self ):

        return json.dumps( self.__dict__ )

    def write_to_file( self ):

        # Generate filename based on first 64 characters of the problem
        filename_base = self.problem[ :64 ]
        # Replace any character that is not alphanumeric or underscore with underscore
        filename_base = "".join( c if c.isalnum() else "-" for c in filename_base )
        # Get the project root directory
        project_root = du.get_project_root()
        # Define the directory where the file will be saved
        directory = f"{project_root}{self.solution_directory}"
        # Get a list of all files that start with the filename base
        existing_files = glob.glob( f"{directory}{filename_base}-*.json" )
        # The count of existing files will be used to make the filename unique
        file_count = len( existing_files )
        # Generate the full file path
        file_path = f"{directory}{filename_base}-{file_count}.json"
        # Print the file path for debugging purposes
        print( f"File path: {file_path}" )

        # Write the JSON string to the file
        with open( file_path, "w" ) as f:
            f.write( self.to_json() )

today     = SolutionSnapshot( problem="what day is today" )
tommorrow = SolutionSnapshot( problem="what day is tommorrow" )
blah      = SolutionSnapshot( problem="i feel so blah today" )

# snapshot.write_to_file()
# snapshot.to_json()

Elapsed time: 232 milliseconds
Elapsed time: 156 milliseconds
Elapsed time: 138 milliseconds


In [23]:
snapshots = [ today, tommorrow, blah ]

for snapshot in snapshots:

    score = today.compare_embedding( snapshot )
    # print( f"Score: {score}" ) Print out both problem values
    print( f"Score: [{score}] for [{snapshot.problem}] == [{today.problem}]" )

Score: [0.9999999187991976] for [what day is today] == [what day is today]
Score: [0.8773281083641067] for [what day is tommorrow] == [what day is today]
Score: [0.7907040929132791] for [i feel so blah today] == [what day is today]
